In [1]:
%load_ext autoreload
%autoreload 2

import math
import utils
import su4ops.quark as Q
import su4ops.elemental as E
import numpy as np
import FiniteVolumeGroups as fvg
from su4ops.constants import NS, gammas

In [2]:


oh = fvg.cubic.Oh()
len(oh.elements)

48

In [3]:
basis = []
fullbasis = []
extraBasis = {}
for s0 in range(0, NS):
  for s1 in range(0, NS):
    for s2 in range(0, NS):
      for s3 in range(0, NS):
        newOp = E.Elemental(
          1, [utils.quark(s0,0), utils.quark(s1,1), utils.quark(s2,0), utils.quark(s3,1)])
        fullbasis.append(newOp)
        foundRelated = False
        for b in basis:
          if newOp.quarks in utils.permutations(b.quarks):
            foundRelated = True
            sign = 1
            if not utils.arePermsEqualParity(newOp.quarks, b.quarks):
              sign = 1
            extraBasis[newOp] = {
              'elemental': E.Elemental(1, b.quarks), 'sign': sign}
        if not foundRelated:
            basis.append(newOp)

In [4]:
print(len(basis), len(extraBasis), len(fullbasis))

100 156 256


In [5]:
rep = []
for g in oh.elements:
  rep.append(utils.makeRepMat(basis, extraBasis, g, oh.elements[0]))


metric = np.matrix([[0. for j in range(len(rep[0]))] for i in range(len(rep[0]))], dtype='complex')
for g in rep:
  metric += np.matmul(np.matrix(g).getH(), np.matrix(g))

In [6]:
for i in range(len(rep[0])):
  for j in range(len(rep[0])):
    if i != j and not np.isclose(metric[i, j], 0):
      print("Off diagonal element is non-zero")
      break
#matPrint(np.array(metric))


fvg.representation_checks.is_valid_rep(rep)

Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero
Off diagonal element is non-zero


AssertionError: rep not closed for elem indices 1 1

In [7]:

for bi, b in enumerate(basis):
  for gi, elem in enumerate(oh.elements):
    dim = len(basis)
    lhs = utils.su4_fullVec_to_reduced(
      b.spatial_rotate(elem), basis, extraBasis)
    rhs = np.zeros(dim, dtype=complex)
    bVec = np.ones(dim)
    for j in range(dim):
      rhs[j] += bVec[j]*rep[gi][j, bi]
    if not np.allclose(lhs, rhs):
      print(lhs)
      print(rhs)
      raise AssertionError(
        "not a valid rep matrix for basis element {} and group element {}".format(bi, gi))


In [8]:
tot = 0
for irrep in oh.elements[0].irreps:
  ops = utils.operators(irrep, rep, oh)
  tot += len(ops)*len(oh.elements[0].irreps[irrep])
  print("{} ops in {}".format(len(ops), irrep))

print("{} operators across all irreps".format(tot))


6 ops in A1g
0 ops in A2u
5 ops in Eg
8 ops in T1u
5 ops in T2g
4 ops in A1u
0 ops in A2g
4 ops in Eu
7 ops in T1g
4 ops in T2u
100 operators across all irreps


In [9]:
for op in utils.operators('A1g', rep, oh):
  utils.print_vec(op, basis)
  print()

(1+0j)*\bar{u}_{a,0}*\bar{d}_{a,0}*\bar{u}_{a,1}*\bar{d}_{a,1}+(-0.5+0j)*\bar{u}_{a,0}*\bar{d}_{a,1}*\bar{u}_{a,0}*\bar{d}_{a,1}+(-0.5+0j)*\bar{u}_{a,1}*\bar{d}_{a,0}*\bar{u}_{a,1}*\bar{d}_{a,0}+

(1+0j)*\bar{u}_{a,0}*\bar{d}_{a,0}*\bar{u}_{a,3}*\bar{d}_{a,3}+(-1+0j)*\bar{u}_{a,0}*\bar{d}_{a,1}*\bar{u}_{a,3}*\bar{d}_{a,2}+(-1+0j)*\bar{u}_{a,1}*\bar{d}_{a,0}*\bar{u}_{a,2}*\bar{d}_{a,3}+(1+0j)*\bar{u}_{a,1}*\bar{d}_{a,1}*\bar{u}_{a,2}*\bar{d}_{a,2}+

(1+0j)*\bar{u}_{a,0}*\bar{d}_{a,1}*\bar{u}_{a,2}*\bar{d}_{a,3}+(-1+0j)*\bar{u}_{a,0}*\bar{d}_{a,1}*\bar{u}_{a,3}*\bar{d}_{a,2}+(-1+0j)*\bar{u}_{a,1}*\bar{d}_{a,0}*\bar{u}_{a,2}*\bar{d}_{a,3}+(1+0j)*\bar{u}_{a,1}*\bar{d}_{a,0}*\bar{u}_{a,3}*\bar{d}_{a,2}+

(1+0j)*\bar{u}_{a,0}*\bar{d}_{a,2}*\bar{u}_{a,1}*\bar{d}_{a,3}+(-0.5+0j)*\bar{u}_{a,0}*\bar{d}_{a,3}*\bar{u}_{a,0}*\bar{d}_{a,3}+(-0.5+0j)*\bar{u}_{a,1}*\bar{d}_{a,2}*\bar{u}_{a,1}*\bar{d}_{a,2}+

(1+0j)*\bar{u}_{a,2}*\bar{d}_{a,0}*\bar{u}_{a,3}*\bar{d}_{a,1}+(-0.5+0j)*\bar{u}_{a,2}*\bar{d}

In [13]:
import copy

def print_op_to_file(vec, basis, filename):
  f = open(filename, "w")

  v = copy.deepcopy(basis)
  for i in range(len(vec)):
    v[i].coef *= vec[i]

  text=""
  for i in range(len(v)):
    if not np.isclose(v[i].coef, 0):
        line = ""
        for q in v[i].quarks:
            line += str(q.spin) + " "
        line += "{} {}".format(v[i].coef.real, v[i].coef.imag)
        #print(line)
        line += "\n"
        text += line
  f.write(text[:-1]) # delete the last newline character
  f.close()


for i,op in enumerate(utils.operators('A1g', rep, oh)):
    print_op_to_file(op, basis,"baryon_udud_A1g_{}_.txt".format(i))
    
for i,op in enumerate(utils.operators('Eg', rep, oh)):
    print_op_to_file(op, basis,"baryon_udud_Eg_{}_.txt".format(i))

In [11]:
def print_nonzero(vec, basis):

  v = copy.deepcopy(basis)
  for i in range(len(vec)):
    v[i].coef *= vec[i]

  text=""
  for i in range(len(v)):
    if not np.isclose(v[i].coef, 0):
        line = ""
        for q in v[i].quarks:
            line += str(q.spin) + " "
        line += str(v[i].coef)
        print(line)

for i,op in enumerate(utils.operators('A1g', rep, oh)):
    print("gamma_{}".format(i))
    print_nonzero(op, basis)
    print("-----------------")


gamma_0
0 0 1 1 (1+0j)
0 1 0 1 (-0.5+0j)
1 0 1 0 (-0.5+0j)
-----------------
gamma_1
0 0 3 3 (1+0j)
0 1 3 2 (-1+0j)
1 0 2 3 (-1+0j)
1 1 2 2 (1+0j)
-----------------
gamma_2
0 1 2 3 (1+0j)
0 1 3 2 (-1+0j)
1 0 2 3 (-1+0j)
1 0 3 2 (1+0j)
-----------------
gamma_3
0 2 1 3 (1+0j)
0 3 0 3 (-0.5+0j)
1 2 1 2 (-0.5+0j)
-----------------
gamma_4
2 0 3 1 (1+0j)
2 1 2 1 (-0.5+0j)
3 0 3 0 (-0.5+0j)
-----------------
gamma_5
2 2 3 3 (1+0j)
2 3 2 3 (-0.5+0j)
3 2 3 2 (-0.5+0j)
-----------------
